<a href="https://colab.research.google.com/github/christine127/Kaggle/blob/main/boston_house_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [112]:
from sklearn.datasets import load_boston
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV,cross_val_score, cross_validate, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error #--squared = 'True' #MSE, squared= 'False' #RMSE

dataset = load_boston()
df = pd.DataFrame(dataset.data, columns = dataset.feature_names)
df['target'] = dataset.target

In [113]:
#전부 다 로그스케일
df= np.log1p(df)
X,y= df.drop('target', axis=1), df['target']

In [83]:
# from scipy.stats import skew
# skewed_features = df.apply(lambda x: skew(x))
# skewed_features = skewed_features[abs(skewed_features)>0.75].index
# print(skewed_features)

In [84]:
# for col in rest_cols:
#   q1,q3 = np.percentile(df[col], [25,75])
#   iqr =q3-q1
#   lower_bound = q1-iqr*1.5
#   upper_bound = q3+ iqr*1.5
#   outliers = df[col][(df[col]<lower_bound)|(df[col]>upper_bound)]
#   print(col, len(outliers))
# df[rest_cols] =std_scaler.fit_transform(df[rest_cols])

In [114]:
##연습을 위해  train,test 분리
X_train,X_test, y_train,y_test = train_test_split(X,y, test_size = 0.2, shuffle=True, random_state= 121)

In [115]:
linear_reg = LinearRegression()
kf = KFold(n_splits = 10, shuffle= False)
cross_val_score(linear_reg,X_train,y_train, cv = kf, scoring= 'neg_root_mean_squared_error')

array([-0.21881323, -0.23067354, -0.16269162, -0.13193866, -0.21629356,
       -0.16451957, -0.14928201, -0.14535854, -0.15958753, -0.19312217])

In [116]:
linear_reg.fit(X_train,y_train)
linear_reg.coef_

array([-0.1202732 , -0.00613582, -0.01015895,  0.16186106, -0.90661023,
        0.50417989,  0.02172324, -0.2283322 ,  0.15229626, -0.15110898,
       -0.5518753 ,  0.05109765, -0.41310506])

In [105]:
lasso = Lasso(alpha= 1.0)
ridge = Ridge(alpha = 1.0)
elsnet = ElasticNet(alpha = 1.0, l1_ratio = 0.2)
rf = RandomForestRegressor() 
gbr = GradientBoostingRegressor() 
xgb = XGBRegressor(objective='reg:squarederror')

In [117]:
models = [lasso, ridge, elsnet, rf, gbr, xgb]
for model in models:
  rmse = cross_val_score(model,X_train,y_train, cv= kf, scoring = 'neg_root_mean_squared_error')
  print(model.__class__.__name__,rmse.mean())


Lasso -0.3906487522495969
Ridge -0.1781835231967844
ElasticNet -0.37125732515164855
RandomForestRegressor -0.14432321523901584
GradientBoostingRegressor -0.13766613214608792
XGBRegressor -0.1420931688674129


In [118]:
st_models = {('rf',rf),('ridge',ridge),('xgb',xgb)}
stacking = StackingRegressor(estimators=st_models, final_estimator= gbr)
rmse = cross_val_score(stacking,X_train,y_train, cv= kf, scoring = 'neg_root_mean_squared_error')
print(stacking.__class__.__name__,rmse.mean()) #-0.20199037211243437

StackingRegressor -0.1517840457605278


In [121]:
gbr.fit(X_train, y_train)
pred = gbr.predict(X_train)

In [124]:
#target 스케일 원래대로 복구
pred = np.expm1(pred)